In [1]:
import pandas as pd
import numpy as np
from m6_feats_comb import *
from m7_utils import *
from m5_models import *
from m3_model_params import lgb_params_2 as lgbm_params
from m3_model_params import xgb_params_2 as xgb_params
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [3]:
INPUT_DIR = 'kaggle/input/linking-writing-processes-to-writing-quality'
FEAT_STORE_DIR = 'feat_store_combined'
train_logs = pd.read_csv(f'{INPUT_DIR}/train_logs.csv')
train_scores = pd.read_csv(f'{INPUT_DIR}/train_scores.csv')
test_logs = pd.read_csv(f'{INPUT_DIR}/test_logs.csv')
ss_df = pd.read_csv(f'{INPUT_DIR}/sample_submission.csv')

logs = pd.concat([train_logs, test_logs], axis=0)
logs = normalise_up_down_times(logs)

In [5]:
# 0.616979

feats = pd.read_pickle('feat_store_combined/base_feats/base_feats.pkl')

train_ids = train_logs.id
test_ids = test_logs.id

# feats_ = pd.read_pickle('feat_store_combined/wpm_feats.pkl')
# feats = feats.merge(feats_, on='id', how='left')

feats_ = pd.read_pickle('feat_store_combined/action_time_gap_by_acti.pkl')
feats = feats.merge(feats_, on='id', how='left')

feats_ = pd.read_pickle('feat_store_combined/action_time_gap.pkl')
feats = feats.merge(feats_, on='id', how='left')

feats = preprocess_feats(feats, MinMaxScaler())
print(feats.shape)

feats = feats.merge(train_scores, on='id', how='left')

train_feats = feats[feats['id'].isin(train_ids)]
test_feats = feats[feats['id'].isin(test_ids)]

 #_, oof_1, rmse, model1 = xgb_cv_pipeline(train_feats=train_feats, 
 #                                        test_feats=test_feats, 
 #                                        xgb_params=xgb_params, 
 #                                        seed=42, 
 #                                        n_repeats=5, 
 #                                        n_splits=6)

n_repeats= 5
n_splits = 6

_, oof_2, rmse, model1 = cv_pipeline(train_feats, 
                                     test_feats, 
                                     lgbm_params, 
                                     lgbm_params['boosting_type'],
                                     seed = 42,
                                     n_repeats= n_repeats,
                                     n_splits = n_splits)

# blend = pd.concat([oof_1, oof_2], axis=0)
# blend_scores = blend.groupby(['id','score'])['prediction'].mean().reset_index()
# blend_rmse = mean_squared_error(blend_scores['score'], blend_scores['prediction'], squared=False)
# print(f'Blend RMSE {blend_rmse:.6f}')

(2474, 457)


/root/Projects/Kaggle/linking-writing/m5_models.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_feats.replace([np.inf, -np.inf], np.nan, inplace=True)


LGBM Average RMSE over 30 folds: 0.622347


In [3]:
train_ids = train_logs.id
test_ids = test_logs.id

n_repeats= 5
n_splits = 6

base_feats = pd.read_pickle(f'{FEAT_STORE_DIR}/base_feats/base_feats.pkl')
base_feats = preprocess_feats(base_feats, StandardScaler())
results = compare_with_baseline_comb(FEAT_STORE_DIR, 
                                     base_feats, 
                                     train_ids, 
                                     test_ids, 
                                     lgbm_params, 
                                     0.623403, 
                                     train_scores,
                                     StandardScaler())

['wpm_feats', 'IKI', 'action_time_gap', 'rep_cut', 'adj_eff_time', 'agg_tmp_feats', 'wc_chage', 'action_time_gap_by_acti']
wpm_feats
Base (2474, 380). New feats (2474, 4). Train feats (2471, 383)
LGBM Average RMSE over 100 folds: 0.618751
Features: wpm_feats. RMSE: 0.618751, Improvement: 0.004652
IKI
Base (2474, 380). New feats (2474, 5). Train feats (2471, 384)
LGBM Average RMSE over 100 folds: 0.619302
Features: IKI. RMSE: 0.619302, Improvement: 0.004101
action_time_gap
Base (2474, 380). New feats (2474, 12). Train feats (2471, 391)
LGBM Average RMSE over 100 folds: 0.619097
Features: action_time_gap. RMSE: 0.619097, Improvement: 0.004306
rep_cut
Base (2474, 380). New feats (2470, 12). Train feats (2471, 391)
LGBM Average RMSE over 100 folds: 0.618806
Features: rep_cut. RMSE: 0.618806, Improvement: 0.004597
adj_eff_time
Base (2474, 380). New feats (2474, 17). Train feats (2471, 396)
LGBM Average RMSE over 100 folds: 0.620630
Features: adj_eff_time. RMSE: 0.620630, Improvement: 0.0027

In [4]:
INPUT_DIR = 'kaggle/input/linking-writing-processes-to-writing-quality'
FEATURE_STORE = 'feat_store_combined'

train_ids = train_logs.id
test_ids = test_logs.id
base_feats = pd.read_pickle(f'{FEAT_STORE_DIR}/base_feats/base_feats.pkl')
base_feats = preprocess_feats(base_feats, StandardScaler())

n_repeats= 5
n_splits = 6

# Usage
max_combination_length = 4 # You can adjust this to test different combination lengths
results_comb = compare_feature_combos(base_dir=FEATURE_STORE, 
                                       base_feats=base_feats,
                                       params=lgbm_params,
                                       baseline_metrics=0.623403,
                                       train_ids=train_ids,
                                       test_ids=test_ids,
                                       train_scores=train_scores,
                                       max_combination_length=max_combination_length,
                                       scaler=StandardScaler())

Number of combinations: 162
Feature set: ('wpm_feats', 'IKI', 'action_time_gap', 'rep_cut')
Base train size: (2474, 380)
(2471, 410) (3, 409)
LGBM Average RMSE over 100 folds: 0.617896
Features: ('wpm_feats', 'IKI', 'action_time_gap', 'rep_cut'). RMSE: 0.617896, Improvement: 0.005507
Feature set: ('wpm_feats', 'IKI', 'action_time_gap', 'adj_eff_time')
Base train size: (2474, 380)
(2471, 415) (3, 414)
LGBM Average RMSE over 100 folds: 0.618362
Features: ('wpm_feats', 'IKI', 'action_time_gap', 'adj_eff_time'). RMSE: 0.618362, Improvement: 0.005041
Feature set: ('wpm_feats', 'IKI', 'action_time_gap', 'agg_tmp_feats')
Base train size: (2474, 380)
(2471, 444) (3, 443)
LGBM Average RMSE over 100 folds: 0.618574
Features: ('wpm_feats', 'IKI', 'action_time_gap', 'agg_tmp_feats'). RMSE: 0.618574, Improvement: 0.004829
Feature set: ('wpm_feats', 'IKI', 'action_time_gap', 'wc_chage')
Base train size: (2474, 380)
(2471, 429) (3, 428)
LGBM Average RMSE over 100 folds: 0.619372
Features: ('wpm_feats

In [6]:
results_comb.sort_values('Metric')

,Feature Combination,Metric,Improvement
80,"(wpm_feats, action_time_gap, action_time_gap_b...",0.616979,0.006424
132,"(wpm_feats, action_time_gap_by_acti)",0.616996,0.006407
60,"(action_time_gap, rep_cut, wc_chage, action_ti...",0.617254,0.006149
138,"(IKI, action_time_gap_by_acti)",0.617273,0.006130
63,"(action_time_gap, adj_eff_time, wc_chage, acti...",0.617365,0.006038
...,...,...,...
61,"(action_time_gap, adj_eff_time, agg_tmp_feats,...",0.621484,0.001919
160,"(wc_chage,)",0.621597,0.001806
122,"(adj_eff_time, agg_tmp_feats, wc_chage)",0.622091,0.001312
98,"(IKI, rep_cut, wc_chage)",0.622324,0.001079


In [3]:
feats = pd.read_pickle('feat_store_combined/base_feats/base_feats.pkl')

feats = preprocess_feats(feats, StandardScaler())
train_ids = train_logs.id
test_ids = test_logs.id

feats = feats.merge(train_scores, on='id', how='left')

train_feats = feats[feats['id'].isin(train_ids)]
test_feats = feats[feats['id'].isin(test_ids)]

 #_, oof_1, rmse, model1 = xgb_cv_pipeline(train_feats=train_feats, 
 #                                        test_feats=test_feats, 
 #                                        xgb_params=xgb_params, 
 #                                        seed=42, 
 #                                        n_repeats=5, 
 #                                        n_splits=6)
                                        
_, oof_2, rmse, model1 = cv_pipeline(train_feats, 
                                     test_feats, 
                                     lgbm_params, 
                                     lgbm_params['boosting_type'],
                                     seed = 42,
                                     n_repeats= 5,
                                     n_splits = 6)

# blend = pd.concat([oof_1, oof_2], axis=0)
# blend_scores = blend.groupby(['id','score'])['prediction'].mean().reset_index()
# blend_rmse = mean_squared_error(blend_scores['score'], blend_scores['prediction'], squared=False)
# print(f'Blend RMSE {blend_rmse:.6f}')

/root/Projects/Kaggle/linking-writing/m5_models.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_feats.replace([np.inf, -np.inf], np.nan, inplace=True)


LGBM Average RMSE over 30 folds: 0.623403


In [ ]:
# BASELINE of base_feats.pkl with LGBM is 0.623403
# 0.608132 - StandardScaler with base + at_activity
# Blend RMSE 0.610875 with tmp_agg I KNEW THEY WERE SHIT

In [7]:
base_feats = pd.read_pickle(f'{FEAT_STORE_DIR}/base_feats/base_feats.pkl')
new_feats = pd.read_pickle(f'{FEAT_STORE_DIR}/IKI.pkl')

feats = base_feats.merge(new_feats, on='id', how='left')
feats = pd.read_pickle(f'{FEAT_STORE_DIR}/base_feats/base_feats.pkl')
feats = preprocess_feats(feats, StandardScaler())

train_ids = train_logs.id
test_ids = test_logs.id

feats = feats.merge(train_scores, on='id', how='left')

train_feats = feats[feats['id'].isin(train_ids)]
test_feats = feats[feats['id'].isin(test_ids)]

 #_, oof_1, rmse, model1 = xgb_cv_pipeline(train_feats=train_feats, 
 #                                        test_feats=test_feats, 
 #                                        xgb_params=xgb_params, 
 #                                        seed=42, 
 #                                        n_repeats=5, 
 #                                        n_splits=6)
                                        
_, oof_2, rmse, model1 = cv_pipeline(train_feats, 
                                     test_feats, 
                                     lgbm_params, 
                                     lgbm_params['boosting_type'],
                                     seed = 42,
                                     n_repeats= 5,
                                     n_splits = 6)


/root/Projects/Kaggle/linking-writing/m5_models.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_feats.replace([np.inf, -np.inf], np.nan, inplace=True)


LGBM Average RMSE over 30 folds: 0.623176


In [3]:
train_ids = train_logs.id
test_ids = test_logs.id

logs = pd.concat([train_logs, test_logs], axis=0)
logs = normalise_up_down_times(logs)

preprocessor = Preprocessor(seed=42)
feats = preprocessor.make_feats(logs)
nan_cols = feats.columns[feats[feats['id'].isin(train_ids)].isna().any()].tolist()
feats = feats.drop(columns=nan_cols)

essays = getEssays(logs)
sent_feats = compute_sentence_aggregations(essays)
par_feats = compute_paragraph_aggregations(essays)
word_feats = create_word_length_features(essays, 'essay', 'id', 'essay_words')
vector_feats = countvectorize_one_one(logs)

feats = feats.merge(sent_feats, how='left', on='id')
feats = feats.merge(par_feats, how='left', on='id')
feats = feats.merge(word_feats, how='left', on='id')
feats = pd.concat([feats, vector_feats], axis=1)

feats.to_pickle('feat_store_combined/base_feats/base_feats.pkl')

Engineering time data
Engineering cursor position data
Engineering word count data
Engineering statistical summaries for features


100%|██████████| 33/33 [01:26<00:00,  2.61s/it, column=word_count_change100, method=kurt]         


Engineering activity counts data


100%|██████████| 2474/2474 [00:00<00:00, 13808.03it/s]
/root/miniconda3/envs/lrp/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Engineering event counts data


...

100%|██████████| 2474/2474 [00:00<00:00, 13864.60it/s]
/root/miniconda3/envs/lrp/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Engineering punctuation counts data


100%|██████████| 2474/2474 [00:00<00:00, 12795.44it/s]


Engineering input words data
Engineering ratios data


/root/Projects/Kaggle/linking-writing/m6_feats_comb.py:418: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feats['word_time_ratio'] = feats['word_count_max'] / feats['up_time_max']
/root/Projects/Kaggle/linking-writing/m6_feats_comb.py:419: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feats['word_event_ratio'] = feats['word_count_max'] / feats['event_id_max']
/root/Projects/Kaggle/linking-writing/m6_feats_comb.py:420: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many t

In [4]:
file_name = 'agg_tmp_feats'
feats = agg_fe_df(logs)
feats.to_pickle(f'{FEAT_STORE_DIR}/{file_name}.pkl')

In [6]:
file_name = 'pause_feats'
feats = pauses_feats(logs)
feats.to_pickle(f'{FEAT_STORE_DIR}/{file_name}.pkl')

In [5]:
file_name = 'adj_eff_time'
feats = process_adjusted_eff_time(logs)
feats.to_pickle(f'{FEAT_STORE_DIR}/{file_name}.pkl')

In [6]:
file_name = 'rep_cut'
feats = process_re_cut_essays(logs)
feats.to_pickle(f'{FEAT_STORE_DIR}/{file_name}.pkl')

100%|██████████| 2470/2470 [00:00<00:00, 4356.68it/s]


In [7]:
file_name = 'action_time_gap'
at_gap = process_feats_action_time_gap(logs)
at_gap.to_pickle(f'{FEAT_STORE_DIR}/{file_name}.pkl')

In [8]:
file_name = 'action_time_gap_by_acti'
feats = process_feats_time_gap_activity(logs)
feats.to_pickle(f'{FEAT_STORE_DIR}/{file_name}.pkl')

In [9]:
file_name = 'IKI'
IKI = logs.groupby(['id']).apply(calculate_pause_features).reset_index()
IKI.to_pickle(f'{FEAT_STORE_DIR}/{file_name}.pkl')

In [4]:
file_name = 'wc_chage'
feats = create_feats_wc_change(logs)
feats.to_pickle(f'{FEAT_STORE_DIR}/{file_name}.pkl')

In [11]:
file_name = 'wpm_feats'
feats = wpm_feats(logs)
feats.to_pickle(f'{FEAT_STORE_DIR}/{file_name}.pkl')

In [ ]:
0.609610
0.611009
0.616839